## Question1 - Toronto Neighbourhood Dataframe

In [34]:
# Installing packages
! pip install pandas

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [35]:
# Imports to first question
from bs4 import BeautifulSoup
import requests
import pandas as pd
# Reading source - Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
# Creating the list
headers="Postalcode, Borough, Neighbourhood"
tableAux=""
for tr in table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    tableAux=tableAux+row1[1:]
# Saving in a file
file=open("AssessmentQ1.csv","wb")
file.write(bytes(tableAux,encoding="ascii",errors="ignore"))
file.close()
# Reading file to create dataframe
df = pd.read_csv('AssessmentQ1.csv',header=None)
df.columns=['Postalcode','Borough','Neighbourhood']
indexNames = df[ df['Borough'] =='Not assigned'].index
df.drop(indexNames , inplace=True)
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
neighbour_df = result.reset_index()
neighbour_df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [36]:
print('Dataframe shape: ',neighbour_df.shape)

Dataframe shape:  (103, 3)


## Question2 - Toronto GeoSpacial Data

In [38]:
# Reading CSV with spatial coordinates
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
# Creating spatial table
geospatial_data = geospatial_data.rename(columns={geospatial_data.columns[0]: 'Postalcode' })
spatial_table = neighbour_df.merge(geospatial_data, on = 'Postalcode')
spatial_table.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [43]:
print('Spatial Dataframe shape: ',spatial_table.shape)

Spatial Dataframe shape:  (103, 5)


## Question3 - Toronto Cluster

In [45]:
# Install package dependences
! pip install geopy
! pip install folium
! pip install matplotlib
! pip install sklearn

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [46]:
# Importation
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
import folium # plotting library

In [50]:
# Creating geolocalization
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [52]:
# Toronto Map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)
#add neighborhood markers to the Toronto map
for lat, long, bor, neigh in zip(spatial_table['Latitude'], spatial_table['Longitude'], 
                                 spatial_table['Borough'], spatial_table['Neighbourhood']):
    label = '{}, {}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker( [lat, long], radius = 7, popup = label, color = 'red', fill = True,
                        fill_color = 'white', fill_opacity = 0.7, parse_html = False).add_to(map_toronto)
map_toronto

In [54]:
# Unifying boroughs
list_boroughs = spatial_table['Borough'].unique()
list_boroughs

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [55]:
# Function to define borough localization
def borough_loc(list_of_places):
    listplaces = []
    for place in list_of_places:
        address = (place + ", Ontario, Canada")
        geolocator = Nominatim(user_agent="TO_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        ##print('{''}, {}, {},'.format(place,latitude,longitude))
        ##listline = [place,latitude,longitude]
        ##listplaces.append(listline)
        listplaces.append(place)
        listplaces.append(latitude)
        listplaces.append(longitude)        
    return listplaces
boroughs = borough_loc(list_boroughs)

In [56]:
import numpy as np
boroughs_df = pd.DataFrame(np.array(boroughs).reshape(11,3), columns = ["Borough","Latitude","Longitude"])
boroughs_df

,Borough,Latitude,Longitude
0,North York,43.7708175,-79.4132998
1,Downtown Toronto,43.655115,-79.380219
2,Queen's Park,43.6599803,-79.3903686
3,Etobicoke,43.67145915,-79.5524920661167
4,Scarborough,43.773077,-79.257774
5,East York,43.6913391,-79.3278212
6,York,44.0007518,-79.4372217
7,East Toronto,43.6247901,-79.3934918
8,West Toronto,43.653963,-79.387207
9,Central Toronto,43.653963,-79.387207


In [57]:
boroughs_df.dtypes
boroughs_df["Latitude"] = pd.to_numeric(boroughs_df["Latitude"])
boroughs_df["Longitude"] = pd.to_numeric(boroughs_df["Longitude"])

In [58]:
# create a map of Toronto
map_toronto_boroughs = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)
#add neighborhood markers to the Toronto map
for lat, long, bor in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], 
                                 boroughs_df['Borough']):
    label = '{}'.format(bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker( [lat, long], radius = 7, popup = label, color = 'red', fill = True, fill_color = 'white',
                        fill_opacity = 0.7, parse_html = False).add_to(map_toronto_boroughs) 
map_toronto_boroughs

In [60]:
# The geographical coordinates for Downtown Toronto
DT_lat = boroughs_df.iloc[5,1]
DT_long = boroughs_df.iloc[5,2]
# The dataframe that contains all the Downtown Toronto neighborhoods
DT_df = spatial_table[spatial_table['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
# create map of Downtown Toronto neighborhoods using latitude and longitude values
map_DT = folium.Map(location=[DT_lat, DT_long], zoom_start=12.4)
# add markers to map
for lat, lng, label in zip(DT_df['Latitude'], DT_df['Longitude'], DT_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker( [lat, lng], radius=5, popup=label, color='red', fill=True, fill_color='#3186cc',
                        fill_opacity=0.7, parse_html=False).add_to(map_DT)
map_DT

In [61]:
# Define Foursquare Credentials and Versions
CLIENT_ID = 'JFHEL2VZIWI5IHKFT552QKLLN5YPDTCZULN3MWVZLG4VJ5I5' # your Foursquare ID
CLIENT_SECRET = 'LHXRE4TO21LX0CITOYCVAF2KEZKQ3RSS5EZHGJYOHFPA44N1' # your Foursquare Secret
VERSION = '20181102' # Foursquare API version

In [62]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
results = requests.get(url).json()

In [63]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [64]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,The Alley,Bubble Tea Shop,43.665922,-79.385567
1,Storm Crow Manor,Theme Restaurant,43.666840,-79.381593
2,DanceLifeX Centre,Dance Studio,43.666956,-79.385297
3,Smith,Breakfast Spot,43.666927,-79.381421
4,Sansotei Ramen 三草亭,Ramen Restaurant,43.666735,-79.385353
5,Ho's Team Barber Shop,Salon / Barbershop,43.665630,-79.381359
6,Fabarnak,Restaurant,43.666377,-79.380964
7,Wickson Social,Gastropub,43.665905,-79.385720
8,DAVIDsTEA,Tea Room,43.664930,-79.380630
9,Glad Day Bookshop,Bookstore,43.665271,-79.380785


In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
                CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'],
                             v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue',
                            'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [66]:
DT_venues = getNearbyVenues(names=DT_df['Neighbourhood'],
                                   latitudes=DT_df['Latitude'],
                                   longitudes=DT_df['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [67]:
print(DT_venues.shape)
DT_venues.head()

(1287, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [68]:
DT_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [262]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
DT_onehot['Neighbourhood'] = DT_venues['Neighbourhood'] 
# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]
DT_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [263]:
DT_onehot.shape

(1287, 208)

In [264]:
DT_grouped = DT_onehot.groupby('Neighbourhood').mean().reset_index()
DT_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.00,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.0000,0.01,0.020000,0.000000,0.000000,0.030000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.000000,0.010000,0.020000,0.010000,0.000000,0.010000,0.000000,0.00,0.01,0.00,0.000000,0.010000,0.000

In [265]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [266]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = DT_grouped['Neighborhood']
for ind in np.arange(DT_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.01,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Burger Joint,Bakery,Cosmetics Shop,Gym,Hotel
1,0.00,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Bakery,Steakhouse,Cheese Shop,Beer Bar,Italian Restaurant
2,0.00,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Plane,Boat or Ferry,Sculpture Garden,Bar,Coffee Shop,Boutique
3,0.00,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Bakery,Café,Breakfast Spot,Caribbean Restaurant
4,0.00,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Salad Place,Ice Cream Shop,Restaurant,Indian Restaurant
5,0.00,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bar,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Noodle House
6,0.00,Grocery Store,Café,Park,Italian Restaurant,Diner,Restaurant,Nightclub,Baby Store,Convenience Store,Coffee Shop
7,0.00,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Mediterranean Restaurant,Hotel,Gym,Gastropub
8,0.00,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Bakery,Steakhouse,Italian Restaurant,Gastropub
9,0.00,Coffee Shop,Café,Hotel,Restaurant,Bakery,American Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Bar


In [267]:
# run K-means to cluster the neighborhoods into 5 clusters
kclusters = 5
DT_grouped_clustering = DT_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0])

In [268]:
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.01,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Burger Joint,Bakery,Cosmetics Shop,Gym,Hotel
1,0.00,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Bakery,Steakhouse,Cheese Shop,Beer Bar,Italian Restaurant
2,0.00,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Plane,Boat or Ferry,Sculpture Garden,Bar,Coffee Shop,Boutique
3,0.00,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Bakery,Café,Breakfast Spot,Caribbean Restaurant
4,0.00,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Salad Place,Ice Cream Shop,Restaurant,Indian Restaurant


In [269]:
'''
#Backup
neighbourhoods_ori = neighbourhoods_venues_sorted
neighbourhoods_ori.head()
'''

'\n#Backup\nneighbourhoods_ori = neighbourhoods_venues_sorted\nneighbourhoods_ori.head()\n'

In [270]:
'''
#Restore
neighbourhoods_venues_sorted = neighbourhoods_ori
neighbourhoods_venues_sorted.head()
'''

'\n#Restore\nneighbourhoods_venues_sorted = neighbourhoods_ori\nneighbourhoods_venues_sorted.head()\n'

In [271]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)
DT_merge = DT_df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
try:
    DT_merge = DT_merge.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
except:
    print('')
DT_merge.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [272]:
neighbourhoods_venues_sorted

,Cluster,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,0.01,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Burger Joint,Bakery,Cosmetics Shop,Gym,Hotel
1,0,0.00,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Bakery,Steakhouse,Cheese Shop,Beer Bar,Italian Restaurant
2,3,0.00,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Plane,Boat or Ferry,Sculpture Garden,Bar,Coffee Shop,Boutique
3,0,0.00,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Bakery,Café,Breakfast Spot,Caribbean Restaurant
4,0,0.00,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Middle Eastern Restaurant,Salad Place,Ice Cream Shop,Restaurant,Indian Restaurant
5,2,0.00,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bar,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Noodle House
6,4,0.00,Grocery Store,Café,Park,Italian Restaurant,Diner,Restaurant,Nightclub,Baby Store,Convenience Store,Coffee Shop
7,0,0.00,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Men's Store,Mediterranean Restaurant,Hotel,Gym,Gastropub
8,0,0.00,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Bakery,Steakhouse,Italian Restaurant,Gastropub
9,0,0.00,Coffee Shop,Café,Hotel,Restaurant,Bakery,American Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Bar


In [273]:
cluster = []
for i in range(0,5):
    select = neighbourhoods_venues_sorted.loc[neighbourhoods_venues_sorted.Cluster==i]
    print(select.size)
    cluster.append(select)

156
12
24
12
12
